## Three Level Models and Intrdoucing Contextual Effects ##

This session introduces the (minor) changes needed to the syntax for the "lmer" command in order to :-

        1) Expand a multilevel model to include more than 2 levels
        2) Include contextual explanatory variables (those measured at level 2 or higher)
        
As in previous sessions, the "foreign" package is needed to import data prepared in different statistical software, and the package "lme4" is needed to conduct the multilevel analysis.

In [2]:
library (foreign)
library (lme4)

Loading required package: Matrix


Data from the European Social Survey (ESS) is typically modelled using 2 level models.  The ESS could be argued to be suitable for a 3 level multilevel model (respondents, within regions, within countries), but there is only very limited regional level data avaliable and it is not always clear how comparable such data are between countries.  For that reason, the following examples are based on the "Achieve" dataset used extensively in the "Multilevel Modelling Using R" by Finch, Bolin and Kelley that features on the course reading list.

This dataset includes data on 10320 students, who are clustered within classes, within schools (hence having a clear 3 level structure).

Models will focus on the variable "geread", which provides a measure of a studnet's reading ability; higher values representing higher levels of achievment.

The data are stored in an SPSS (.sav) datafile.  Therefore the "read.spss" command from the "foreign" package is sued to import the dataset.

In [3]:
Achieve <- read.spss ("Achieve.sav")

N.B the models created in this section are intended to demonstrate how the syntax for 3 level models varies from that for 2 level models.  No consideration is given to the substantive interpretation of teh results, and no attempt has been made to recode variables to improve model fit/estimation.

### Creating a 3 Level VPC (null) Model ###

The syntax below creates a 2 level null model, identical to those created in lab session 2.  This provides a benchmark against which to compare the subsuquent 3 level model.  For practice, can you calculate the vpc for the two level model shown below?

In [ ]:
ModelA_2lev <- lmer (geread~1+(1|school), data=Achieve)
summary (ModelA_2lev)

Expanding the above model to be a 3 level model (students within classes, within schools) is simply a case of adding the additional clustering variable "class" to the random part of the  model.  Cllustering variables should decrease in size as you move from left to right and are sepearted by a "/", e.g in the command below random intercepts are requested at both class and school levels, and classes are seen as clustered within schools 

In [ ]:
ModelA_3lev <- lmer (geread~1+(1|school/class), data=Achieve)
summary (ModelA_3lev)

As in previous sessions, the "anova" command can be used to compare the model fit of two models.  In the case of the command below the fit of the null model with 2 levels is compared to the equivilent 3 level model.  What do you conclude about whether adding the class level to the model has improved fit with the data?

Can you calculate VPC values for the school and class levels?

In [ ]:
anova (ModelA_2lev, ModelA_3lev)

### 3 Level Random Intercept Model ###

The syntax below creates a 3 level random intercept model.  The explanatory variables included are :-

        1) gevocab    - student's score on a vocabulary test
        2) clenroll   - size of student's reading classroom
        3) cenroll    - size of student's school

In [4]:
ModelA_3lev.ri <- lmer (geread~1+gevocab+clenroll+cenroll+gender +(1|school/class), data=Achieve)
summary (ModelA_3lev.ri)

Warning message:
“Some predictor variables are on very different scales: consider rescaling”

Linear mixed model fit by REML ['lmerMod']
Formula: 
geread ~ 1 + gevocab + clenroll + cenroll + gender + (1 | school/class)
   Data: Achieve

REML criterion at convergence: 43135.3

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2169 -0.5668 -0.2081  0.3187  4.4780 

Random effects:
 Groups       Name        Variance Std.Dev.
 class:school (Intercept) 0.09045  0.3007  
 school       (Intercept) 0.07638  0.2764  
 Residual                 3.69824  1.9231  
Number of obs: 10320, groups:  class:school, 568; school, 160

Fixed effects:
              Estimate Std. Error t value
(Intercept)  1.648e+00  2.157e-01   7.642
gevocab      5.077e-01  8.434e-03  60.203
clenroll     1.893e-02  9.557e-03   1.981
cenroll     -3.706e-06  3.640e-06  -1.018
gender       1.820e-02  3.819e-02   0.477

Correlation of Fixed Effects:
         (Intr) gevocb clnrll cenrll
gevocab  -0.130                     
clenroll -0.924 -0.063              
cenroll  -0.132  0.025 -0.007       
gender   -0.26

As with 2 level models, comparing the random intercept model to the equivilent vpc model can help establish if the independent variables introduced to the model have any explanatory power.  What might you conclude in this case?

In [ ]:
anova (ModelA_3lev,ModelA_3lev.ri)

### 3 Level Random Slope Models ###

As with a two level model, random slope models are created by including the relevant explanatory variable in both the fixed, and random, parts of the formula.  For example, the command below introduces random slops for the variable "gevocad", the effect of which is allowed to vary both between classes within a school and between schools.

In [ ]:
ModelA_3lev.rs1 <- lmer (geread~1+gevocab+clenroll+cenroll+(1+gevocab|school/class), data=Achieve)
summary (ModelA_3lev.rs1)

It is possible to allow different explanatory varaibles to have random slopes at different levels within the model.  This is achieved by spliting the random part of the mdoel formula so that each level is detailed sepeartely.  Consider the command below, rather than reading,

        "(1+gevocab|school/class)"

the random part of the model now reads,

        "(1|school)+(1+gevocab|class)"

this provides a random intercept at both the class and school levels, but the variable "gevocab" only has a random slope at the class level.

In [ ]:
ModelA_3lev.rs2 <- lmer (geread~1+gevocab+clenroll+cenroll+(1|school)+(1+gevocab|class), data=Achieve)
summary (ModelA_3lev.rs2)

### Further Practice ###

In the random slope models run above, R has provided a warning that some of the explanatory variables appear to be measured on very different scales.  You could try looking at the descriptive statistics for the explanatory variables and then recoding/rescaling, before rerunning the random slope models to see if the concern is resolved.  Do you changes alter your substantive findings?

Alternatively, the dataset contains teh variable "corp" which represents the school district in which each school is located.  Can you expand the syntax to run four level models with students, nested within classes, within schools, within districts?  Try running a null model, a random intercept model and a random slope model.

### Adding Contextual Variables and Cross-level Interactions to Multilevel Models ###

All the multilevel models fitted in the previous practical sessions have involved only Level 1 explanatory variables (i.e. respondent characteristics in the European Social Survey).  Explanatory variables can also be included at higher levels of the model for instance, in the example above, the variable "clenroll"was measured at the class level and "cenroll" measured at the School level.  Interactions between variables at different levels can also be used to investiagte how the impact of a lower level explaatory variable changes depending on the context in which it occurs.  For example, does the impact of education on indivdual hedonism vary depending on overall educational achievemnet within the country?

The interpretation of contexual variables, and cross-level interactions, is broadly similar to interpreting regression coefficients and interaction terms in single level regression models.  The following examples are taken from materials created by the Centre for Multilevel Modelling at te University of Bristol and illustrate (briefly) how to run and interpret such models.

### Introduction to the Scottish Youth Cohort Trends Dataset###

In this section, models will be constructed using data from the Scottish School Leavers Survey (SSLS), a nationally representative survey of young people. The data come from seven cohorts of young people collected in the first sweep of the study, carried out at the end of the final year of compulsory schooling (aged 16-17) when most sample members had taken Standard grades.

The dependent variable is a total attainment score. Each subject is graded on a scale from 1 (highest) to 7 (lowest) and, after recoding so that a high numeric value denotes a high grade, the total is taken across subjects. The analysis dataset contains the student-level variables considered in Module 3 together with a school identifier and three school-level variables:

     caseid     - Anonymised student identifier
     schoolid   - Anonymised school identifier
     score      - Point score calculated from awards in Standard grades taken at age 16. Scores range from 0 to 75, with 
                a higher score indicating a higher attainment
     cohort90   - The sample includes the following cohorts: 1984, 1986, 1988, 1990, 1996 and 1998. The cohort90 variable is 
                calculated by subtracting 1990 from each value. Thus values range from -6 (corresponding to 1984) to 
                8 (1998), with 1990 coded as zero
     female     - Sex of student (1 = female, 0 = male)
     sclass     - Social class, defined as the higher class of mother or father (1 = managerial and professional, 
                2 = intermediate, 3 = working, 4 = unclassified)
     schtype    - School type, distinguishing independent schools from state-funded schools 
                (1 = independent, 0 = state-funded)
     schurban   - Urban-rural classification of school (1 = urban, 0 = town or rural)
     schdenom   - School denomination (1 = Roman Catholic, 0 = non-denominational)
     sclass1    - Dummy variable for managerial and professional social class
     sclass2    - Dummy variable for intermediate social class
     sclass3    - Dummy variable for unclassified social class

There are 33,988 students in 508 schools.

As in previous sessions the "foreign" pckage is used to import the data.

In [ ]:
schdata <- read.dta ("school.dta")

Each school-level variable is binary, so descriptive statistics are best considered in terms of the proportion in each category. This can be done by using the "table", "prop.table" and "cumsum" commands to produce respectively the table of frequencies, percentages and cumulative percentages.  First restrict the scope of the command to one record per school by using the "unique" command restricted to the columns of interest.

In [ ]:
schdata_un <- unique(schdata[, c(2, 7, 8, 9)])

#### School Type ####

In [ ]:
cbind(Freq=table(schdata_un$schtype), 
      Perc=prop.table(table(schdata_un$schtype)), 
      Cum=cumsum(prop.table(table(schdata_un$schtype))))

#### Urban or Rural Setting ####

In [ ]:
cbind(Freq=table(schdata_un$schurban), 
      Perc=prop.table(table(schdata_un$schurban)), 
      Cum=cumsum(prop.table(table(schdata_un$schurban))))

#### School Denomination ####

In [ ]:
cbind(Freq=table(schdata_un$schdenom), 
      Perc=prop.table(table(schdata_un$schdenom)),
      Cum=cumsum(prop.table(table(schdata_un$schdenom))))

The following proportion of schools are in category 1 of each variable: schtype (10% independent), schurban (68% urban), and schdenom (16% Catholic).

### A Baseline Random Slope Model ###

Consider the following model which contains the student's cohort, gender and social class as fixed effects along with a random effect for cohort (allowing it to vary across schools).

𝐬𝐜𝐨𝐫𝐞𝑖𝑗  = 𝛽0 + 𝛽1𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎𝑖𝑗 + 𝛽2𝐟𝐞𝐦𝐚𝐥𝐞𝑖𝑗 + 𝛽3𝐬𝐜𝐥𝐚𝐬𝐬𝟏𝑖𝑗 + 𝛽4𝐬𝐜𝐥𝐚𝐬𝐬𝟐𝑖𝑗 + 𝛽5𝐬𝐜𝐥𝐚𝐬𝐬𝟒𝑖𝑗 +𝑢0𝑗 + 𝑢1𝑗𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎𝑖𝑗 + 𝑒𝑖𝑗

In terms of structure this is the same as the random slope models introduced in Week 3.

The syntax for this model is shown below.  It is identical to that introduced in the practical session on random slopes except for the argument "lmerControl(optimizer = "bobyqa")".  This argument involves an alternative optimisation routine which can lead to faster estimation of models in particular contexts.  Those interested in the details will find more in the documentation for "lme4".

In [ ]:
fit1 <- lmer(score ~ cohort90 + female + sclass1 + sclass2 + sclass4 + (1 + cohort90 | schoolid), 
              data = schdata, REML = FALSE, lmerControl(optimizer = "bobyqa"))
summary(fit1)

### Adding Contextual Effects ###

Adding school type ("schtype") to the model is simply a case of adding the variable name to the list of fixed effects, as shown in the command below that creates  a new model stored in the object "fit2".  In other words, explantory variables at higher levels are entered into the command exactly as level 1 variables were.

In [ ]:
fit2 <- lmer(score ~ cohort90 + female + sclass1 + sclass2 + sclass4 + schtype + (1 + cohort90 | schoolid), 
              data = schdata, REML = FALSE, lmerControl(optimizer = "bobyqa"))
summary(fit2)

A child in an independent school would be expected to have a score that is 4.25 points higher than a child in a state school (from the same cohort, and of the same sex and social background). This effect is strongly statistically significant because the estimated coefficient is more than 5 times its standard error.

There has also been a slight reduction in the school-level variance. After accounting for school type, the between-school variance for the 1990 cohort (the intercept variance) reduces from 22.5 to 20.6. However, there remains a large amount of unexplained between-school variance.

Next the urban-rural indicator of school location can be added to the model.

One way of doing this would be to write the model out in full,

fit2 <- lmer(score ~ cohort90 + female + sclass1 + sclass2 + sclass4 + schtype + schurban + (1 + cohort90 | schoolid), 
              data = schdata, REML = FALSE, lmerControl(optimizer = "bobyqa"))
summary(fit2)

An alternative is shown in the command box below.  Where a new model is the same as an earlier model but now including additional variables, it is possible to use the "update" command.  The command below creates a new model, stored in the object "fit3" which is the same as the previous model but also includes the variable "schurban"

Updating a previous model can make for easier documenting of a research process since the differences between models are more explicit within the syntax. 

In [ ]:
fit3 <- update(fit2, . ~ . + schurban)
summary (fit3)

On average, a student in an urban school has a score that is 1.44 points lower than a student attending a school in a town or rural area. This difference is adjusted for the effects of school type, and student cohort, gender and social class. The between-school variance in 1990 has decreased further but by a very small amount (from 20.6 to 20.0).

Using te command box below, update model "fit3" to include the variable "schdenom" as a fixed effect.  What do you conclude about the impact of attending a catholic school compared to a non-denominational school?

In [ ]:
fit4 <- update(fit3, . ~ . + schdenom)
summary(fit4)

Assuming the model ran correctly, you should have concluded that the ratio of the estimated coefficient of "schdenom" to its standard error is less than 0.3, so there is little evidence of a difference between Catholic and non-denominational schools.

The model stored in "fit3" could therefore be considerd an appropraite model to take forward for further investigation.

### Cross-level Interactions ###

Thus far, from model "fit3", it seems student attainment at age 16 is significantly related to the year in which the exams were taken (cohort), and student gender and parental social class. At the school level, there are differences in student attainment between independent and state schools, and between urban and rural schools. However, we have considered only main effects of these variables. 

In practice, the relationship between 𝑦 and an explanatory variable 𝑥1 may depend on the value of another variable 𝑥2, i.e. an interaction effect between 𝑥1 and 𝑥2. In a multilevel model, 𝑥1 and 𝑥2 may be defined at the same or different levels. If they are at different levels, the interaction is referred to as a cross-level interaction.

Consider the possible interaction between cohort (level 1) and school type (level 2), i.e the following model,

𝐬𝐜𝐨𝐫𝐞𝑖𝑗  = 𝛽0 + 𝛽1𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎𝑖𝑗 + 𝛽2𝐟𝐞𝐦𝐚𝐥𝐞𝑖𝑗 + 𝛽3𝐬𝐜𝐥𝐚𝐬𝐬𝟏𝑖𝑗 + 𝛽4𝐬𝐜𝐥𝐚𝐬𝐬𝟐𝑖𝑗 + 𝛽5𝐬𝐜𝐥𝐚𝐬𝐬𝟒𝑖𝑗  +𝛽6𝐬𝐜𝐡𝐭𝐲𝐩𝐞𝑗 + 𝛽7𝐬𝐜𝐡𝐮𝐫𝐛𝐚𝐧𝑗 + 𝛽8𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎𝐗𝐬𝐜𝐡𝐭𝐲𝐩𝐞𝑖𝑗
+𝑢0𝑗 + 𝑢1𝑗𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎𝑖𝑗 + 𝑒𝑖𝑗

where the variable "cohort90Xschtype" is the interaction term between cohort and school type.

The interaction terms is created by multiplying together the two original variables, and can then be added to the model via the "update" command.

In [ ]:
schdata$cohort90Xschtype <- schdata$cohort90*schdata$schtype

fit5 <- update(fit3, . ~ . + cohort90Xschtype)
summary(fit5)

The estimated coefficient of the interaction variable "cohort90Xschtype" is almost 6 times its standard error, so this is strong evidence that the effect of cohort differs for independent and state schools. (Equivalently,  the difference between independent and state schools differs across cohorts.) However, the addition of this interaction effect does little to explain between-school differences in attainment trends: the school-level variance in the "cohort90" coefficient has reduced only slightly from 0.148 to 0.138.

To see the nature of the interaction effect, consider the fixed part of the model that contains "cohort90" and "schtype":

    1.213 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎+5.291 𝐬𝐜𝐡𝐭𝐲𝐩𝐞−0.599 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎𝐗𝐬𝐜𝐡𝐭𝐲𝐩𝐞

For schtype=0 (state schools), this equation reduces to:

    1.213 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎
    
So in the average state school (i.e. with 𝑢1𝑗=0), the average year-on-year increase in attainment is 1.213 points.

For schtype=1 (independent schools), this equation reduces to:

    1.213 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎+5.291−0.599 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎=(1.213−0.599) 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎+5.291=0.614 𝐜𝐨𝐡𝐨𝐫𝐭𝟗𝟎+5.291

So in the average independent school, the average year-on-year increase in attainment is 0.614 points.

The coefficient of schtype (estimated as 5.291) is the expected difference in attainment between independent and state schools in 1990 (i.e. when cohort90= 0).

Overall, mean attainment is higher in independent schools than in state schools, but independent schools experienced a smaller increase in attainment with cohort.